In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import warnings
def noop(*args, **kwargs): pass
warnings.warn = noop

In [3]:
import copy
from collections import ChainMap
from multiprocessing import cpu_count

In [4]:
from IPython.display import display
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.externals.joblib import Parallel, delayed
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm

In [5]:
from basedir import SAMPLE
from utils import from_feather, to_feather

In [6]:
seed = 1
np.random.seed(seed)

In [7]:
class Flatten(nn.Module):
    """Converts N-dimensional tensor into 'flat' one."""

    def __init__(self, keep_batch_dim=True):
        super().__init__()
        self.keep_batch_dim = keep_batch_dim

    def forward(self, x):
        if self.keep_batch_dim:
            return x.view(x.size(0), -1)
        return x.view(-1)

In [8]:
class Classifier(nn.Module):
    def __init__(self, n_in, conv=(32, 64, 128, 256, 512), fc=(512, 512, 256), n_out=9):
        super().__init__()
        layers = []
        fi = n_in
        for size in conv:
            layers += [
                nn.Conv1d(fi, size, 3),
                nn.BatchNorm1d(size),
                nn.LeakyReLU(0.05)]
            fi = size
        layers.append(nn.AdaptiveAvgPool1d(1))
        layers.append(Flatten())
        for size in fc:
            layers += [
                nn.Linear(fi, size),
                nn.BatchNorm1d(size),
                nn.LeakyReLU(0.05),
                nn.Dropout(0.15)]
            fi = size
        layers.append(nn.Linear(fi, n_out))
        self.layers = nn.ModuleList(layers)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [9]:
def accuracy(y_true, y_pred):
    n = len(y_true)
    y_hat = y_pred.reshape(9, n).argmax(axis=0)
    value = (y_true == y_hat).mean()
    return 'accuracy', value, True

In [10]:
from torch.optim.lr_scheduler import _LRScheduler

class CyclicLR(_LRScheduler):
    
    def __init__(self, optimizer, schedule, last_epoch=-1):
        assert callable(schedule)
        self.schedule = schedule
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        return [self.schedule(self.last_epoch, lr) for lr in self.base_lrs]

In [11]:
def cosine(t_max, eta_min=0):
    
    def scheduler(epoch, base_lr):
        t = epoch % t_max
        return eta_min + (base_lr - eta_min)*(1 + np.cos(np.pi*t/t_max))/2
    
    return scheduler

In [12]:
ID_COLS = ['series_id', 'measurement_number']

In [43]:
def create_grouped_array(data, group_col='series_id', drop_cols=ID_COLS):
    X_grouped = np.row_stack([
        group.drop(columns=drop_cols).values[None]
        for _, group in data.groupby(group_col)])
    return X_grouped.transpose(0, 2, 1)

In [44]:
def create_datasets(X, y, test_size=0.2, dropcols=ID_COLS):
    enc = LabelEncoder()
    y_enc = enc.fit_transform(y)
    X_grouped = create_grouped_array(X)
    X_train, X_valid, y_train, y_valid = train_test_split(X_grouped, y_enc, test_size=0.1)
    X_train, X_valid = [torch.tensor(arr, dtype=torch.float32) for arr in (X_train, X_valid)]
    y_train, y_valid = [torch.tensor(arr, dtype=torch.long) for arr in (y_train, y_valid)]
    train_ds = TensorDataset(X_train, y_train)
    valid_ds = TensorDataset(X_valid, y_valid)
    return train_ds, valid_ds, enc

In [14]:
def create_test_dataset(X, dropcols=ID_COLS):
    X_grouped = np.row_stack([
        group.drop(columns=dropcols).values[None]
        for _, group in X.groupby('series_id')])
    X_grouped = torch.tensor(X_grouped.transpose(0, 2, 1)).float()
    y_fake = torch.tensor([0] * len(X_grouped)).long()
    return TensorDataset(X_grouped, y_fake)

In [15]:
def create_loaders(train_ds, valid_ds, bs=512, jobs=0):
    train_dl = DataLoader(train_ds, bs, shuffle=True, num_workers=jobs)
    valid_dl = DataLoader(valid_ds, bs, shuffle=False, num_workers=jobs)
    return train_dl, valid_dl

In [16]:
def accuracy(output, target):
    return (output.argmax(dim=1) == target).float().mean().item()

In [17]:
x_trn, y_trn, x_tst = from_feather('x_trn', 'y_trn', 'x_tst')

In [18]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

In [19]:
lr = 1e-2
wd = 1e-5
bs = 400
n_epochs = 1000
patience = 100
no_improvements = 0
jobs = 12
best_loss = np.inf
best_acc = 0
best_weights = None
history = []
lr_history = []

trn_ds, val_ds, enc = create_datasets(x_trn, y_trn['surface'])
trn_dl, val_dl = create_loaders(trn_ds, val_ds, bs, jobs=jobs)
dataset_sizes = {'train': len(trn_ds), 'val': len(val_ds)}

model = Classifier(10)
model.to(device)
criterion = nn.CrossEntropyLoss(reduction='sum')
optimizer = optim.RMSprop(model.parameters(), lr=lr, weight_decay=wd)
iterations_per_epoch = len(trn_dl)
scheduler = CyclicLR(optimizer, cosine(t_max=iterations_per_epoch * 2, eta_min=lr/100))

for epoch in range(n_epochs):
    stats = {'epoch': epoch + 1, 'total': n_epochs}
    
    for phase, loader in (('train', trn_dl), ('val', val_dl)):
        training = phase == 'train'
        running_loss = 0.0
        num_correct = 0
        num_examples = 0
        n_batches = 0
        
        for batch in loader:
            x_batch, y_batch = [b.to(device) for b in batch]
            optimizer.zero_grad()
        
            # compute gradients only during 'train' phase
            with torch.set_grad_enabled(training):
                outputs = model(x_batch)
                loss = criterion(outputs, y_batch)
                
                # don't update weights and rates when in 'val' phase
                if training:
                    scheduler.step()
                    loss.backward()
                    optimizer.step()
                    lr_history.extend(scheduler.get_lr())
                    
            running_loss += loss.item()
            num_correct += (outputs.argmax(dim=1) == y_batch).sum().item()
            num_examples += len(x_batch)
            
        epoch_loss = running_loss / dataset_sizes[phase]
        stats[phase] = epoch_loss
        
        # early stopping: save weights of the best model so far
        if phase == 'val':
            val_acc = num_correct/num_examples
            # if epoch_loss < best_loss:
            if val_acc > best_acc:
                # print('loss improvement on epoch: %d' % (epoch + 1))
                print('accuracy improvement on epoch: %d' % (epoch + 1))
                best_acc = val_acc
                best_loss = epoch_loss
                best_weights = copy.deepcopy(model.state_dict())
                torch.save(best_weights, 'best_weights.pth')
                no_improvements = 0
            else:
                no_improvements += 1
            stats['val_acc'] = val_acc
                
    history.append(stats)
    print('[{epoch:03d}/{total:03d}] '
          'train: {train:.4f} - '
          'val: {val:.4f} - '
          'val. acc: {val_acc:2.2%}'.format(**stats))
    
    if no_improvements >= patience:
        print('early stopping after epoch {epoch:03d}'.format(**stats))
        break

if best_weights is not None:
    # print(f'Loading the best weights with the training loss: {best_loss:.4f}')
    print(f'Loading the best weights with the validation accuracy: {best_acc:2.2%}')
    model.load_state_dict(best_weights)

accuracy improvement on epoch: 1
[001/1000] train: 2.2650 - val: 1.7709 - val. acc: 30.97%
accuracy improvement on epoch: 2
[002/1000] train: 1.6682 - val: 1.6052 - val. acc: 37.80%
accuracy improvement on epoch: 3
[003/1000] train: 1.6563 - val: 1.5750 - val. acc: 40.16%
accuracy improvement on epoch: 4
[004/1000] train: 1.5265 - val: 1.5203 - val. acc: 44.09%
[005/1000] train: 1.6318 - val: 1.4842 - val. acc: 41.73%
accuracy improvement on epoch: 6
[006/1000] train: 1.4218 - val: 1.4392 - val. acc: 46.72%
accuracy improvement on epoch: 7
[007/1000] train: 1.5578 - val: 1.4493 - val. acc: 49.08%
[008/1000] train: 1.3835 - val: 1.4069 - val. acc: 47.51%
[009/1000] train: 1.4783 - val: 1.4185 - val. acc: 48.03%
accuracy improvement on epoch: 10
[010/1000] train: 1.3502 - val: 1.3730 - val. acc: 50.39%
[011/1000] train: 1.4132 - val: 1.3935 - val. acc: 49.61%
accuracy improvement on epoch: 12
[012/1000] train: 1.3053 - val: 1.3655 - val. acc: 51.97%
[013/1000] train: 1.3726 - val: 1.3307

[127/1000] train: 0.5253 - val: 0.7588 - val. acc: 74.54%
[128/1000] train: 0.3597 - val: 0.7798 - val. acc: 73.23%
[129/1000] train: 0.5423 - val: 0.8167 - val. acc: 74.80%
accuracy improvement on epoch: 130
[130/1000] train: 0.3655 - val: 0.7770 - val. acc: 75.33%
[131/1000] train: 0.4612 - val: 0.7704 - val. acc: 75.33%
accuracy improvement on epoch: 132
[132/1000] train: 0.3397 - val: 0.7435 - val. acc: 77.95%
[133/1000] train: 0.5380 - val: 0.7947 - val. acc: 72.97%
[134/1000] train: 0.3530 - val: 0.7329 - val. acc: 74.54%
accuracy improvement on epoch: 135
[135/1000] train: 0.4418 - val: 0.7249 - val. acc: 79.00%
[136/1000] train: 0.3354 - val: 0.7326 - val. acc: 76.64%
[137/1000] train: 0.4679 - val: 0.8224 - val. acc: 73.23%
[138/1000] train: 0.3338 - val: 0.7316 - val. acc: 78.22%
[139/1000] train: 0.5340 - val: 0.7683 - val. acc: 73.23%
[140/1000] train: 0.3441 - val: 0.7492 - val. acc: 76.12%
[141/1000] train: 0.4991 - val: 0.7952 - val. acc: 74.80%
[142/1000] train: 0.3263 

[264/1000] train: 0.1145 - val: 0.7365 - val. acc: 81.89%
[265/1000] train: 0.2371 - val: 0.7690 - val. acc: 80.05%
[266/1000] train: 0.1103 - val: 0.6617 - val. acc: 81.63%
[267/1000] train: 0.1649 - val: 0.8066 - val. acc: 79.53%
[268/1000] train: 0.1124 - val: 0.6981 - val. acc: 81.89%
[269/1000] train: 0.1441 - val: 0.7121 - val. acc: 80.84%
[270/1000] train: 0.0919 - val: 0.7699 - val. acc: 81.10%
[271/1000] train: 0.2045 - val: 1.0037 - val. acc: 76.64%
[272/1000] train: 0.1234 - val: 0.8145 - val. acc: 79.00%
[273/1000] train: 0.2794 - val: 0.8269 - val. acc: 78.48%
[274/1000] train: 0.1411 - val: 0.6632 - val. acc: 81.36%
[275/1000] train: 0.2335 - val: 0.7925 - val. acc: 80.31%
[276/1000] train: 0.1252 - val: 0.7789 - val. acc: 80.58%
[277/1000] train: 0.1445 - val: 0.8515 - val. acc: 77.43%
[278/1000] train: 0.1151 - val: 0.8016 - val. acc: 80.05%
[279/1000] train: 0.2502 - val: 0.8360 - val. acc: 79.79%
[280/1000] train: 0.1225 - val: 0.7740 - val. acc: 80.84%
[281/1000] tra

[405/1000] train: 0.2525 - val: 0.8302 - val. acc: 79.79%
[406/1000] train: 0.0839 - val: 0.7504 - val. acc: 81.10%
[407/1000] train: 0.1295 - val: 0.7997 - val. acc: 82.94%
[408/1000] train: 0.0655 - val: 0.7312 - val. acc: 83.99%
[409/1000] train: 0.0870 - val: 0.9212 - val. acc: 78.74%
[410/1000] train: 0.0618 - val: 0.7544 - val. acc: 80.84%
[411/1000] train: 0.0980 - val: 1.0551 - val. acc: 79.00%
[412/1000] train: 0.0680 - val: 0.8884 - val. acc: 80.05%
[413/1000] train: 0.0680 - val: 0.8675 - val. acc: 81.10%
[414/1000] train: 0.0423 - val: 0.9059 - val. acc: 80.05%
[415/1000] train: 0.2648 - val: 0.6686 - val. acc: 82.68%
[416/1000] train: 0.0666 - val: 0.6693 - val. acc: 82.15%
[417/1000] train: 0.1062 - val: 0.7406 - val. acc: 82.15%
[418/1000] train: 0.0509 - val: 0.7383 - val. acc: 81.89%
[419/1000] train: 0.1382 - val: 0.8361 - val. acc: 80.84%
[420/1000] train: 0.0677 - val: 0.7736 - val. acc: 82.68%
early stopping after epoch 420
Loading the best weights with the validat

In [29]:
model = Classifier(10).to(device)
model.load_state_dict(torch.load('best_weights.pth'))

In [30]:
model.layers = model.layers[:-1]

In [32]:
_ = model.eval()

In [51]:
def prepare_data(data, device):
    return torch.tensor(create_grouped_array(data)).float().to(device)

In [56]:
def extract_features(X, weights_file, device='cpu'):
    model = Classifier(10).to(device)
    model.load_state_dict(torch.load(weights_file))
    model.layers = model.layers[:-1]
    model.eval()
    input_tensor = prepare_data(X, device)
    feat = model(input_tensor)
    return feat.detach()

In [45]:
# trn_tensor = torch.tensor(create_grouped_array(x_trn)).float().to(device)

In [ ]:
for data, filename in ((x_trn, 'trn_feat.npy'), (x_tst, 'tst_feat.npy')):
    feat = extract_features(data, 'best_weights.pth')
    np.save(filename, feat.numpy())

In [ ]:
tst_feat = model(prepare_data(x_tst))

In [23]:
test_ds = create_test_dataset(x_tst)

In [26]:
_, _, enc = create_datasets(x_trn, y_trn['surface'])

In [27]:
test_results = []
model.eval()
for x_batch, _ in DataLoader(test_ds, batch_size=1000, shuffle=False):
    output = model(x_batch.to(device))
    test_results += output.argmax(dim=1).tolist()

In [28]:
submit = pd.read_csv(SAMPLE)
submit['surface'] = enc.inverse_transform(test_results)
submit.to_csv('submit.csv', index=None)
!kaggle c submit career-con-2019 -f 'submit.csv' -m "Conv1d deeper (ensure eval mode)"

100%|██████████████████████████████████████| 51.9k/51.9k [00:00<00:00, 47.3kB/s]
Successfully submitted to CareerCon 2019 - Help Navigate Robots 